In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [16]:
#load in json
df = pd.read_json('data/elden_ring_text.json')




In [ ]:
print("myack myack myack")


In [18]:
#create a new df with all rows with accessories not null
columns = ['accessories', 'arts', 'gems', 'goods', 'protector', 'weapon', 'dialog']

#if name_en
df_accessories = df['accessories'].dropna()
df_arts = df['arts'].dropna()

df_crafting = df['gems'].dropna()
df_goods = df.dropna()
df_protector = df['protector'].dropna()
df_weapons = df['weapon'].dropna()
df_dialog = df['dialog'].dropna()


In [ ]:
df_weapons = df['weapon'].dropna()
df_weapons.tail()

In [20]:

jp_columns = ['name_jp', 'info_jp', 'caption_jp', 'effect_jp', 'id']

def normalizeClean(df, column):
    new_df = pd.json_normalize(df[column])
    #delete and columns with 'jp' in the name
    
    new_df = new_df.drop_duplicates().drop(columns=jp_columns, errors='ignore')
    #add a columns with the name of the column
    new_df['type'] = column
    

    #add a column 
    return new_df

df_accessories = normalizeClean(df, 'accessories')
df_arts = normalizeClean(df, 'arts')
df_goods = normalizeClean(df, 'goods')
df_protector = normalizeClean(df, 'protector')
df_weapons = normalizeClean(df, 'weapon')
df_dialog = normalizeClean(df, 'dialog')
dfs = [df_accessories, df_arts, df_goods, df_protector, df_weapons, df_dialog]


In [ ]:
df_weapons.tail(10)

In [22]:
#clean all text of any html tags or special/escape characters and numbers 
def cleanText(df):
    for column in df.columns:
        df[column] = df[column].str.replace(r'<[^>]*>', '')
        df[column] = df[column].str.replace(r'&[^;]*;', '')
        df[column] = df[column].str.replace(r'[0-9]', '')
        df[column] = df[column].str.replace('\n', ' ')
    return df

for df in dfs:
    cleanText(df)


In [ ]:
df_dialog.head(10)
type(df_dialog['info_en'][0])


In [ ]:
import re
chunk_data = []
pattern = re.compile(r'\[(\d+)\]\s*(.*)')


for index, row in df_dialog.iterrows():
    if type(row['info_en']) is not str:
        continue
    info = row['info_en']
    name = row['name_en']
    chunks = info.split('<br/><br/>')
    count = 1
    for chunk in chunks:
        match = pattern.match(chunk.strip())
        if match:
            text = match.group(2)
            if name == "End of Game Dialog + Extra Morgott dialog":
                name = "End of Game Dialog"
            chunk_data.append({'name_en': name + str(count), 'text': text, 'type': 'dialog'})

        count += 1

print(chunk_data)

df_dialog_chunked = pd.DataFrame(chunk_data)
df_dialog_chunked.head(10)

In [ ]:
df_weapons.tail(10)

In [ ]:

dfs = [df_accessories, df_arts, df_goods, df_protector, df_weapons]

#join all the dataframes together on name_en
df_all = pd.concat(dfs)
#if there is coption_en and info_en, combine them

df_all['info_en'] = df_all['info_en'].fillna('')
df_all['caption_en'] = df_all['caption_en'].fillna('')
df_all['effect_en'] = df_all['effect_en'].fillna('')


df_all[df_all['name_en'] == 'Flaming Bolt']


In [27]:
df_all['text'] = df_all['caption_en'] + ' ' + df_all['info_en'] + ' ' + df_all['effect_en']



df_all = pd.concat([df_all, df_dialog_chunked])
df_all = df_all[['name_en', 'text', 'type']]
#drop any rows with no name_en
df_all = df_all.dropna(subset=['name_en'])

#reset index
df_all = df_all.reset_index(drop=True)

#rename name_en to name
df_all = df_all.rename(columns={'name_en': 'name'})

df_all.to_pickle('data/cleaned_elden_ring_data.pkl')

In [28]:
df_all.head(10)
#make into a json file row by row
df_all.to_json('data/cleaned_elden_ring_data.json', orient='records')
